In [1]:

import acquire_ry
import ry_prepare
import ry_wrangle


import pandas as pd
import matplotlib as plt
import numpy as np
import time


import unicodedata
import re
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

In [2]:
# full wrangle test
train_exp, X_train, y_train, X_validate, y_validate, X_test, y_test = ry_wrangle.wrangle_github(cached=True)

acquiring data
preparing data
splitting data
complete
X-train shape (234, 6) X_validate shape (79, 6) X_test shape (79, 6)


In [5]:
check = pd.read_json('readmes2.json')

In [6]:
check.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 827 entries, 0 to 826
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   language  827 non-null    object
 1   content   827 non-null    object
dtypes: object(2)
memory usage: 19.4+ KB


In [3]:
train_exp.head()

,language,content,clean,stemmed,lemmatized,words,doc_length
26,HTML,Environmental_Data_Analytics\nData analytics c...,environmentaldataanalytics data analytics cour...,environmentaldataanalyt data analyt cours duke...,environmentaldataanalytics data analytics cour...,"[environmentaldataanalytics, data, analytics, ...",74
36,Java,MinCED - Mining CRISPRs in Environmental Datas...,minced mining crisprs environmental datasets m...,minc mine crispr environment dataset minc prog...,minced mining crisprs environmental datasets m...,"[minced, mining, crisprs, environmental, datas...",219
35,Python,Open Simulation Interface (OSI)\n\nThe Open Si...,open simulation interface osi open simulation ...,open simul interfac osi open simul interfac 1 ...,open simulation interface osi open simulation ...,"[open, simulation, interface, osi, open, simul...",259
156,Python,Community Water Model (CWatM)\nIIASA\n13rd Oct...,community water model cwatm iiasa 13rd october...,commun water model cwatm iiasa 13rd octob 2020...,community water model cwatm iiasa 13rd october...,"[community, water, model, cwatm, iiasa, 13rd, ...",245
546,HTML,EnvironmentalGame\nThis game was developed for...,environmentalgame game developed ' environment...,environmentalgam game develop ' environ scienc...,environmentalgame game developed ' environment...,"[environmentalgame, game, developedenvironment...",9


In [4]:
# check balance of dataset for under/over sampling
labels = pd.concat([train_exp.language.value_counts(), # get total counts of languages
                    train_exp.language.value_counts(normalize=True)], axis=1) # getting the prop of languages
labels.columns = ['n', 'percent']
labels
# balance is not bad, will not need to over/under sample

,n,percent
Python,83,0.354701
JavaScript,62,0.264957
HTML,49,0.209402
Java,40,0.170940


In [8]:
# create a list of words for each language category
python_words = ' '.join(train_exp[train_exp.language=='Python'].lemmatized)
js_words = ' '.join(train_exp[train_exp.language=='JavaScript'].lemmatized)
html_words = ' '.join(train_exp[train_exp.language=='HTML'].lemmatized)
java_words = ' '.join(train_exp[train_exp.language=='Java'].lemmatized)

# remove single letter words to reduce noise
python_words = re.sub(r'\s.\s', '', python_words)
js_words = re.sub(r'\s.\s', '', js_words)
html_words = re.sub(r'\s.\s', '', html_words)
java_words = re.sub(r'\s.\s', '', java_words)


In [9]:
# get the count of words by category
python_words = pd.Series(python_words.split()).value_counts()
js_words = pd.Series(js_words.split()).value_counts()
html_words = pd.Series(html_words.split()).value_counts()
java_words = pd.Series(java_words.split()).value_counts()


In [13]:
python_words
# note: 4 most common = '00', 'file', 'data', 'environmental'

00                                        226
file                                      204
data                                      183
environmental                             172
using                                     121
                                         ... 
curlhttpskiznichegithubiomycodoinstall      1
20150415                                    1
3990                                        1
preseason                                   1
akvatiske                                   1
Length: 6986, dtype: int64

In [14]:
js_words
# note: 'data', 'file', 'environmental' are in top 5 words

data             131
file              97
system            81
project           77
environmental     76
                ... 
describes          1
lgpl               1
ssh                1
container          1
subscribe          1
Length: 4333, dtype: int64

In [15]:
html_words
# note: 'data', 'file', 'environmental' are top 3 words

data               66
file               50
environmental      49
worcester          47
nd                 46
                   ..
variantaberrant     1
teaching            1
shadow              1
glance              1
subscribe           1
Length: 2498, dtype: int64

In [16]:
java_words
# note: 'data', 'file', 'environmental' are in top 5 words

property          103
&#9;               85
environmental      48
file               44
data               44
                 ... 
dataconfig          1
contructed          1
screenshots         1
recommend           1
webinfclassess      1
Length: 1808, dtype: int64